In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
!conda create --name nemo python==3.10.12

In [ ]:
!conda update -n base -c conda-forge conda

In [ ]:
!conda init bash

In [ ]:
!source activate nemo

In [ ]:
!apt-get install sox libsndfile1 ffmpeg
!pip install wget text-unidecode matplotlib>=3.3.2

In [ ]:
git clone https://github.com/NVIDIA/NeMo-text-processing
cd NeMo-text-processing
./reinstall.sh dev

In [ ]:
apt-get update && apt-get install -y libsndfile1 ffmpeg
git clone https://github.com/NVIDIA/NeMo
cd NeMo
./reinstall.sh dev

# VITS and NeMo

VITS is a neural network that converts text characters into an audio sample. For more details on the model, please refer to Nvidia's [VITS Model Card](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/tts_en_lj_vits), or the original [paper](https://arxiv.org/abs/2106.06103).


# Training

Now that we looked at the VITS model, let's see how to train a VITS Model



In [10]:
# NeMo's training scripts are stored inside the examples/ folder. Let's grab the vits.py file
# as well as the vits.yaml file
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/vits.py
!(mkdir -p conf \
  && cd conf \
  && wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/conf/vits.yaml \
  && cd ..)

--2023-10-12 21:06:50--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/vits.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1216 (1.2K) [text/plain]
Saving to: ‘vits.py’

vits.py             100%[===================>]   1.19K  --.-KB/s    in 0s      

2023-10-12 21:06:50 (42.6 MB/s) - ‘vits.py’ saved [1216/1216]

--2023-10-12 21:06:50--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/conf/vits.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5599 (5.5K) [text/plain]
Saving to: ‘vits.yaml’


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
while True:pass

In [ ]:
python manifest_vits.py --data-root /content/drive/MyDrive/ --val-size 0.1 --test-size 0.2

In [ ]:
%%bash
source activate nemo

python
from nemo.collections.tts.models import VitsModel
model = VitsModel.load_from_checkpoint("/content/drive/MyDrive/TTS_checkpoint_VITS/VITS/2023-10-11_10-13-46/checkpoints/VITS21.ckpt")
model.save_to(save_path="mymodel.nemo")

In [ ]:
CUDA_VISIBLE_DEVICES=0 python vits.py model.sample_rate=16000 train_dataset=/content/drive/MyDrive/datasetTTS_malavoglia/train_manifest_vits.json validation_datasets=/content/drive/MyDrive/datasetTTS_malavoglia/val_manifest_vits.json trainer.strategy='ddp_find_unused_parameters_true' trainer.check_val_every_n_epoch=5 exp_manager.exp_dir=/content/drive/MyDrive/TTS_checkpoint_VITS/finetune +init_from_plt_ckpt=/content/drive/MyDrive/TTS_checkpoint_VITS/VITS/2023-10-11_10-13-46/checkpoints/VITS21.ckpt

In [ ]:
CUDA_VISIBLE_DEVICES=0 python vits.py model.sample_rate=16000 train_dataset=/content/drive/MyDrive/datasetTTS_malavoglia/train_manifest_vits.json validation_datasets=/content/drive/MyDrive/datasetTTS_malavoglia/val_manifest_vits.json trainer.strategy='ddp_find_unused_parameters_true' trainer.check_val_every_n_epoch=5 exp_manager.exp_dir=/content/drive/MyDrive/TTS_checkpoint_VITS/finetune +init_from_nemo_model=./mymodel.nemo

In [ ]:
CUDA_VISIBLE_DEVICES=0 python vits.py model.sample_rate=16000 train_dataset=/content/drive/MyDrive/datasetTTS_malavoglia/train_manifest_vits.json validation_datasets=/content/drive/MyDrive/datasetTTS_malavoglia/val_manifest_vits.json trainer.strategy='ddp_find_unused_parameters_true' trainer.check_val_every_n_epoch=5 exp_manager.exp_dir=/content/drive/MyDrive/TTS_checkpoint_VITS/finetune

# Training Data

In order to train VITS, it is highly recommended to obtain high quality speech data with the following properties:
  - Sampling rate of 22050Hz or higher
  - Speech should contain a variety of speech phonemes
  - Audio split into segments of 1-10 seconds
  - Audio segments should not have silence at the beginning and end
  - Audio segments should not contain long silences inside

After obtaining the speech data and splitting into training, validation, and test sections, it is required to construct .json files to tell NeMo where to find these audio files.

The .json files should adhere to the format required by the `nemo.collections.tts.data.dataset.TTSDataset` class. For example, here is a sample .json file

```json
{"audio_filepath": "/path/to/audio1.wav", "text": "the transcription", "duration": 0.82}
{"audio_filepath": "/path/to/audio2.wav", "text": "the other transcription", "duration": 2.1}
...
```
Please note that the duration is in seconds.


## Evaluating VITS

Let's evaluate the quality of the VITS model.

VITS is end-to-end model, so we don't need any additional models to generate audios.

In [ ]:
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import IPython.display as ipd
import numpy as np
import torch
import librosa
import soundfile as sf

target_sr = 16000

audio_path = "/content/drive/MyDrive/datasetTTS_malavoglia/wavs/imalavoglia_00_verga_f000008.wav"
text_raw = "Il meccanismo delle passioni che la determinano in quelle basse sfere è meno complicato	e potrà quindi osservarsi con maggior precisione."


audio_data, orig_sr = sf.read(audio_path)
if orig_sr != target_sr:
    audio_data = librosa.core.resample(audio_data, orig_sr=orig_sr, target_sr=target_sr)

# Let's double-check that everything matches up!
print(f"Duration (s): {len(audio_data)/target_sr}")
print("Transcript:", text_raw)
ipd.Audio(audio_data, rate=target_sr)

In [ ]:
from nemo.collections.tts.models import VitsModel

In [ ]:
model = torch.load('/content/drive/MyDrive/TTS_checkpoint_VITS/VITS/2023-10-11_10-13-46/checkpoints/VITS--loss_gen_all=21.6121-epoch=279.ckpt',map_location ='cpu')
#model_ = VitsModel.load_from_checkpoint("/content/drive/MyDrive/TTS_checkpoint_VITS/VITS/2023-10-11_10-13-46/checkpoints/VITS--loss_gen_all=21.6121-epoch=279.ckpt").eval()

In [ ]:
tokens = model.parse(text_raw)
audio_pred = model.convert_text_to_waveform(tokens=tokens).cpu().detach().numpy()

print("predicted audio")
ipd.Audio(audio_pred, rate=target_sr)

In [ ]:
audio_to_mel = model.audio_to_melspec_processor


len_audio = torch.tensor(len(audio_data)).view(1, -1)

spec_pred, _ = audio_to_mel(torch.tensor(audio_pred).view(1, -1), len_audio)
spec_orig, _ = audio_to_mel(torch.tensor(audio_data).view(1, -1), len_audio)

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(spec_orig[0][0].cpu().detach())
ax[1].imshow(spec_pred[0][0].cpu().detach())

ax[0].set_title('Original spectrogram')
ax[1].set_title('Predicted spectrogram')
fig.show()

In [ ]:
while True:pass